# 0. Setup
Import dependencies.

In [1]:
import json
from datetime import date
from datetime import datetime
from dateutil.relativedelta import relativedelta
import requests
from config import api_key

import pandas as pd
from sqlalchemy import create_engine

# 1. Initialize Variables
Initialize variables used in making API calls to TMDB.

## URLs
- `discover_base_url`: Base URL of TMDB's Discover API to search movies by different types of data. This includes the API key.
- `movie_base_url`: Base URL of TMDB's Movie API to search details of a specific movie by ID.
- `discover_target_url`: Full URL of TMDB's Discover API which includes TMDB properties to filter movies by. Example URL: https://api.themoviedb.org/3/discover/movie?api_key={api_key}&release_date.gte=2010-01-01&release_date.lte=2010-01-31&region=us&with_release_type=2|3&vote_count.gte=1&sort_by=popularity.desc&page=1
- `movie_target_url`: Full URL of TMDB's Movie API which includes the TMDB movie. Example URL: https://api.themoviedb.org/3/movie/550?api_key={api_key}

## TMDB Properties
- `release_date.gte={release_start_date}`: Earliest date of movies' release dates to search for using TMDB's Discover API.
- `release_date.lte={today}`: Latest date of movies' release dates to search for using TMDB's Discover API.

## Other
- `today`: Today's date.

**Hard-Coded TMDB URL Properties**:
- `region=US`: ISO 3166-1 code to filter movies' release dates by. Set specifically to *US*.
- `with_release_type=2|3`: Type of releases. *2|3* pulls movies with theatrical releases.
- `vote_count.gte=1`: Minimum number of votes by TMDB users on movies to search for in TMDB's Discover API. Set to *1* to exclude movies with 0 votes, reducing overall search results.
- `sort_by=popularity.desc`: Sort movies pulled through TMDB's Discover API by [TMDB's popularity metric](https://developers.themoviedb.org/3/getting-started/popularity) among TMDB users, descending.
- `append_to_results=credits`: Add to the result of an API call more details. This is used when making an API call to TMDB's Movie API to also add details on cast/crew of the movie.

In [2]:
today = date.today()

discover_base_url = f"https://api.themoviedb.org/3/discover/movie?api_key={api_key}"
movie_base_url = "https://api.themoviedb.org/3/movie/"

# Initialize start date as 10 years from today. Keep first day of month.
release_start_date = date((today - relativedelta(years = 10)).year, today.month, 1)
region = "US"

# Initialize variables of empty lists to populate upon iterating through APIs.
movies_lst = []
directors_lst = []
actors_lst = []

# Initialize variables of arbitrary Unique IDs to later utilize for actors and directors.
actor_uid = 1
director_uid = 1

# 2. Request Data from API
Make calls to TMDB's APIs to retrieve and store data into lists.

In [4]:
# Print the range of dates within which to pull movies data from.
print(f"Request: {str(release_start_date)} to {str(today)}")

# Loop through the first 500 pages of the Discover TMDB API result. Maximum pages allowed is 500.
for page in range(1, 501):
    # Append TMDB properties, including page number, to the base URL of TMDB's Discover API.
    discover_target_url = f"{discover_base_url}&release_date.gte={str(release_start_date)}&release_date.lte={str(today)}&region=US&with_release_type=2|3&vote_count.gte=1&sort_by=popularity.desc&page={page}"

    # Retrieve 'results' from TMDB's Discover API.
    discover_response = requests.get(discover_target_url, timeout=None)
    discover_data = discover_response.json()
    results = discover_data['results']

    # Print which page the API call was made for.
    print(f"Page {page} of 500:")
    
    # For each movie in 'results' of TMDB's Discover API, make a call to TMDB's Movie API, check for criteria, and add data to lists if met.
    for movie in results:
        # Store the movie's TMDB ID.
        id = movie["id"]
        # Append ID, API key, and 'append_to_response=credits' to the base URL of TMDB's Movie API.
        movie_target_url = f"{movie_base_url}{id}?api_key={api_key}&append_to_response=credits"

        # Retrieve data from TMDB's Movie API of the particular movie.
        movie_response = requests.get(movie_target_url, timeout=None)
        movie_data = movie_response.json()
        # Store the movie's release date in a variable.
        movie_release_date = datetime.strptime(movie_data['release_date'], '%Y-%m-%d').date()

        # Check the movie's data to see whether its data should be added to our lists. If not, skip and move on to the next movie.
        if (movie_data['budget'] == 0 and movie_data['revenue'] == 0): # Skip if the movie's budget and revenue are both $0.
            print(f"{id} ({movie_data['title']}) has both $0 budget and revenue. Skipping...")
            continue
        elif movie_data['original_language'] != 'en': # Skip if the movie's original language is not English.
            print(f"{id} ({movie_data['title']}) is not originally in English. Skipping...")
            continue
        elif (movie_release_date < release_start_date or movie_release_date > today): # Skip if the movie's release date doesn't fall between the designated start date and today.
            print(f"{id} ({movie_data['title']}) was released outside of the {release_start_date} to {today} range. Skipping...")
            continue
        else: # Append movie data to our lists.
            movies_lst.append({
                "id": id,
                "title": movie_data['title'],
                "tagline": movie_data['tagline'],
                "overview": movie_data['overview'],
                "homepage": movie_data['homepage'],
                "release_date": movie_data['release_date'],
                "budget": movie_data['budget'],
                "revenue": movie_data['revenue'],
                "popularity": movie_data['popularity'],
                "runtime": movie_data['runtime'],
                "poster_path": movie_data['poster_path']
            })
            
            for actor in movie_data['credits']['cast']:
                if (actor['known_for_department'] == "Acting" and actor['character'] != ""):
                    actors_lst.append({
                        "id": actor_uid,
                        "actor_id": actor['id'],
                        "name": actor['name'],
                        "profile_path": actor['profile_path'],
                        "movie_id": id,
                        "character": actor['character']
                    })
                    
                    actor_uid += 1

            for member in movie_data['credits']['crew']:
                if member['job'] != 'Director':
                    continue
                elif member['job'] == 'Director':
                    directors_lst.append({
                        "id": director_uid,
                        "director_id": member['id'],
                        "name": member['name'],
                        "profile_path": member['profile_path'],
                        "movie_id": id
                    })
                    
                    director_uid += 1
                    break

            print(f"{id} ({movie_data['title']}) was successfully added!")

Request: 2013-01-01 to 2023-01-15
Page 193 of 500:
669664 (My Donkey, My Lover & I) has both $0 budget and revenue. Skipping...
284711 (Sharktopus vs. Pteracuda) has both $0 budget and revenue. Skipping...
680813 (Flee) is not originally in English. Skipping...
454650 (Beast of Burden) has both $0 budget and revenue. Skipping...
433630 (Inconceivable) has both $0 budget and revenue. Skipping...
136403 (Populaire) has both $0 budget and revenue. Skipping...
136278 (Blood) has both $0 budget and revenue. Skipping...
329010 (Emelie) has both $0 budget and revenue. Skipping...
500850 (The Silence of Others) has both $0 budget and revenue. Skipping...
382127 (Manifesto) was successfully added!
761388 (Lady of the Manor) has both $0 budget and revenue. Skipping...
393732 (Mean Dreams) has both $0 budget and revenue. Skipping...
337154 (The Meddler) was successfully added!
168055 (Love Is in the Air) is not originally in English. Skipping...
340613 (The Wife) was successfully added!
197624 (A

451499 (School of Life) has both $0 budget and revenue. Skipping...
566872 (The Burnt Orange Heresy) has both $0 budget and revenue. Skipping...
389630 (All We Had) has both $0 budget and revenue. Skipping...
300605 (City of God: 10 Years Later) has both $0 budget and revenue. Skipping...
177566 (Jack the Giant Killer) was successfully added!
158914 (The Look of Love) has both $0 budget and revenue. Skipping...
925943 (Saint Omer) has both $0 budget and revenue. Skipping...
225044 (Like Sunday, Like Rain) was successfully added!
1019446 (Wildcat) has both $0 budget and revenue. Skipping...
606876 (Rifkin's Festival) has both $0 budget and revenue. Skipping...
634904 (Inherit the Viper) has both $0 budget and revenue. Skipping...
104329 (Last Passenger) has both $0 budget and revenue. Skipping...
Page 200 of 500:
215776 (Madras Cafe) is not originally in English. Skipping...
524789 (The Devil's Doorway) has both $0 budget and revenue. Skipping...
186976 (The Golden Dream) is not origina

343701 (Bunny the Killer Thing) has both $0 budget and revenue. Skipping...
279968 (White Settlers) has both $0 budget and revenue. Skipping...
122662 (Mardock Scramble: The Third Exhaust) has both $0 budget and revenue. Skipping...
Page 206 of 500:
793992 (Three Months) has both $0 budget and revenue. Skipping...
243935 (Rob the Mob) was successfully added!
655437 (Poupelle of Chimney Town) has both $0 budget and revenue. Skipping...
641668 (Seance) has both $0 budget and revenue. Skipping...
256962 (Little Boy) was successfully added!
1019391 (Retrograde) has both $0 budget and revenue. Skipping...
499537 (The Wolf House) is not originally in English. Skipping...
1032696 (The Subtle Art of Not Giving a #@%!) has both $0 budget and revenue. Skipping...
258947 (Deadly Virtues: Love. Honour. Obey.) has both $0 budget and revenue. Skipping...
776660 (Cryptozoo) has both $0 budget and revenue. Skipping...
926378 (#Unknown) has both $0 budget and revenue. Skipping...
399056 (Sunset) is not

Page 212 of 500:
449757 (I Am Not a Witch) has both $0 budget and revenue. Skipping...
209416 (Supermensch: The Legend of Shep Gordon) has both $0 budget and revenue. Skipping...
403368 (Havenhurst) has both $0 budget and revenue. Skipping...
532647 (Trespassers) has both $0 budget and revenue. Skipping...
333675 (Band of Robbers) has both $0 budget and revenue. Skipping...
258216 (Nymphomaniac: Vol. I) was successfully added!
362136 (Prem Ratan Dhan Payo) is not originally in English. Skipping...
589049 (The Photograph) was successfully added!
550506 (Saint Judy) has both $0 budget and revenue. Skipping...
174682 (Raze) has both $0 budget and revenue. Skipping...
180296 (They Came Together) has both $0 budget and revenue. Skipping...
152736 (Child of God) has both $0 budget and revenue. Skipping...
877619 (The Moderator) has both $0 budget and revenue. Skipping...
190469 (Redirected) was successfully added!
375509 (Corpse Party) has both $0 budget and revenue. Skipping...
124623 (I Kn

77221 (Black Gold) was released outside of the 2013-01-01 to 2023-01-15 range. Skipping...
413857 (Tragedy Girls) has both $0 budget and revenue. Skipping...
393834 (Goldstone) has both $0 budget and revenue. Skipping...
157099 (Better Living Through Chemistry) was successfully added!
316410 (Bound) has both $0 budget and revenue. Skipping...
308027 (People, Places, Things) has both $0 budget and revenue. Skipping...
464431 (The Wilding) has both $0 budget and revenue. Skipping...
230158 (Aziz Ansari: Buried Alive) has both $0 budget and revenue. Skipping...
127847 (7 Boxes) has both $0 budget and revenue. Skipping...
276935 (Ek Villain) is not originally in English. Skipping...
414027 (Jasper Jones) has both $0 budget and revenue. Skipping...
385736 (Kicks) has both $0 budget and revenue. Skipping...
265228 (Timbuktu) is not originally in English. Skipping...
421848 (Recovery) has both $0 budget and revenue. Skipping...
249688 (The End of the Tour) was successfully added!
Page 219 of 

...

42257 (Hester Street) has both $0 budget and revenue. Skipping...
587928 (Violentia) has both $0 budget and revenue. Skipping...
471084 (It's Gawd!) has both $0 budget and revenue. Skipping...
327308 (Elenore Makes Love) has both $0 budget and revenue. Skipping...
226576 (Likeness) has both $0 budget and revenue. Skipping...
794461 (Nemesis) has both $0 budget and revenue. Skipping...
653568 (The Evening Hour) has both $0 budget and revenue. Skipping...
260536 (666: Kreepy Kerry) was successfully added!
250651 (The Case Against 8) has both $0 budget and revenue. Skipping...
586421 (WBCN and the American Revolution) has both $0 budget and revenue. Skipping...
447380 (Along for the Ride) has both $0 budget and revenue. Skipping...
320412 (Camp Massacre) has both $0 budget and revenue. Skipping...
172542 (Go for Sisters) has both $0 budget and revenue. Skipping...
447420 (Better Than Love) has both $0 budget and revenue. Skipping...
293271 (Moomins on the Riviera) is not originally in Eng

233380 (Behind The Fifty Shades) has both $0 budget and revenue. Skipping...
168864 (GMO OMG) has both $0 budget and revenue. Skipping...
1022128 (Don't Look at the Demon) was successfully added!
489996 (Clara's Ghost) has both $0 budget and revenue. Skipping...
214328 (Slink) has both $0 budget and revenue. Skipping...
717501 (Quarantine Cat Film Festival) was successfully added!
259303 (The Poisoning) was successfully added!
211952 (Women Who Kill) has both $0 budget and revenue. Skipping...
820875 (Call Time The Finale) has both $0 budget and revenue. Skipping...
669770 (Broil) has both $0 budget and revenue. Skipping...
666321 (Boris and the Bomb) has both $0 budget and revenue. Skipping...
541342 (The Riot Act) has both $0 budget and revenue. Skipping...
510656 (Softness of Bodies) has both $0 budget and revenue. Skipping...
93103 (The Last Gladiators) has both $0 budget and revenue. Skipping...
826754 (The Swearing Jar) has both $0 budget and revenue. Skipping...
430124 (Allegian

Page 488 of 500:
469105 (Tie the Knot) has both $0 budget and revenue. Skipping...
340616 (Baden Baden) has both $0 budget and revenue. Skipping...
790227 (Ghost Light) has both $0 budget and revenue. Skipping...
572281 (The Favorite) was successfully added!
763569 (Moments Like This Never Last) has both $0 budget and revenue. Skipping...
349087 (Beautiful Something) has both $0 budget and revenue. Skipping...
876502 (Asteroid) has both $0 budget and revenue. Skipping...
669384 (Acquitted by Faith) was successfully added!
459427 (Car Dogs) was successfully added!
293787 (From Here To Eternity) has both $0 budget and revenue. Skipping...
287958 (Immediately Afterlife) has both $0 budget and revenue. Skipping...
749139 (Android Uprising) has both $0 budget and revenue. Skipping...
474002 (Bernard and Huey) has both $0 budget and revenue. Skipping...
819926 (Hellbox) has both $0 budget and revenue. Skipping...
757169 (Boy Meets Boy) has both $0 budget and revenue. Skipping...
263775 (ETXR

149232 (The Pardon) was successfully added!
1006967 (Jim Crockett Promotions: Ric Flair's Last Match) has both $0 budget and revenue. Skipping...
695086 (Atticus v. the Architect: The Political Assassination of Don Siegelman) has both $0 budget and revenue. Skipping...
Page 494 of 500:
449100 (Morsefest 2015: Question Mark) has both $0 budget and revenue. Skipping...
380683 (Mad) has both $0 budget and revenue. Skipping...
672816 (Cicada) has both $0 budget and revenue. Skipping...
542994 (Jane and Emma) has both $0 budget and revenue. Skipping...
537734 (The Weekend) has both $0 budget and revenue. Skipping...
124069 (Far Out Isn't Far Enough: The Tomi Ungerer Story) has both $0 budget and revenue. Skipping...
459970 (Don't Come Back from the Moon) has both $0 budget and revenue. Skipping...
357837 (Dancin' - It's On!) has both $0 budget and revenue. Skipping...
350175 (COMIX: Beyond the Comic Book Pages) has both $0 budget and revenue. Skipping...
203059 (John Hodgman: RAGNAROK) has 

859350 (Hold Your Fire) has both $0 budget and revenue. Skipping...
778629 (King Car) has both $0 budget and revenue. Skipping...
773848 (Near Enemies) has both $0 budget and revenue. Skipping...
735495 (Loves Me, Loves Me Not) has both $0 budget and revenue. Skipping...
729293 (Vigilante) was successfully added!
727281 (Nowhere) has both $0 budget and revenue. Skipping...
Page 500 of 500:
709798 (Blood Immortal) has both $0 budget and revenue. Skipping...
700544 (Cry for the Bad Man) has both $0 budget and revenue. Skipping...
657011 (Circumstantial Pleasures) has both $0 budget and revenue. Skipping...
648715 (Fat Chance) has both $0 budget and revenue. Skipping...
647672 (How Dogs Got Their Shapes) has both $0 budget and revenue. Skipping...
640094 (90 Feet from Home) has both $0 budget and revenue. Skipping...
623098 (Pearl) has both $0 budget and revenue. Skipping...
611744 (Deep Space) has both $0 budget and revenue. Skipping...
604279 (The Kids from 62-F) has both $0 budget and 

In [5]:
# Check how many total movies were added to 'movies_lst'.
print(f"Total number of movies added: {len(movies_lst)}")

Total number of movies added: 2858


# 3. Load Lists into DataFrames
- Store lists of movies, directors, and actors into DataFrames.
- Save JSON files of movies, directors, and actors.

In [6]:
movies_df = pd.DataFrame(movies_lst)
movies_df = movies_df.drop_duplicates(subset=['id'])
movies_df.head()

,id,title,tagline,overview,homepage,release_date,budget,revenue,popularity,runtime,poster_path
0,315162,Puss in Boots: The Last Wish,Say hola to his little friends.,Puss in Boots discovers that his passion for a...,https://www.dreamworks.com/movies/puss-in-boot...,2022-12-21,90000000,208171553,10610.921,103,/1NqwE6LP9IEdOZ57NCT51ftHtWT.jpg
1,76600,Avatar: The Way of Water,Return to Pandora.,Set more than a decade after the events of the...,https://www.avatar.com/movies/avatar-the-way-o...,2022-12-16,460000000,1752000000,3427.076,192,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg
2,899112,Violent Night,You better watch out.,When a team of mercenaries breaks into a wealt...,https://www.violentnightmovie.com,2022-11-30,20000000,73000000,2342.145,112,/1XSYOP0JjjyMz1irihvWywro82r.jpg
3,536554,M3GAN,Friendship has evolved.,A brilliant toy company roboticist uses artifi...,https://www.m3ganmovie.com,2023-01-06,12000000,52580135,2199.890,102,/7CNCv9uhqdwK7Fv4bR4nmDysnd9.jpg
4,436270,Black Adam,The world needed a hero. It got Black Adam.,"Nearly 5,000 years after he was bestowed with ...",https://www.dc.com/BlackAdam,2022-10-21,200000000,389000000,1922.130,125,/pFlaoHTZeyNkG83vxsAJiGzfSsa.jpg


In [7]:
movies_df.to_csv('Data/movies.csv')

In [8]:
directors_df = pd.DataFrame(directors_lst)
directors_df = directors_df.drop_duplicates(subset=['id', 'movie_id'])
directors_df.head()

,id,director_id,name,profile_path,movie_id
0,1,1450348,Joel Crawford,/3F60mZB4sGgB51jAIglwn6OrUgB.jpg,315162
1,2,2710,James Cameron,/9NAZnTjBQ9WcXAQEzZpKy4vdQto.jpg,76600
2,3,76927,Tommy Wirkola,/lhvD0FltdzJqrI5xr5IN90nyh6f.jpg,899112
3,4,1288796,Gerard Johnstone,/cdvpfGKHLchtOtoN8gtVWhSHabN.jpg,536554
4,5,59521,Jaume Collet-Serra,/saAeTFfgt9YRAybXmzfmIz01UCB.jpg,436270


In [9]:
directors_df.to_csv('Data/directors.csv')

In [10]:
actors_df = pd.DataFrame(actors_lst)
actors_df.head()

,id,actor_id,name,profile_path,movie_id,character
0,1,3131,Antonio Banderas,/79kz5KdLmpPlugIHVSS1cZ4kGZA.jpg,315162,Puss in Boots (voice)
1,2,3136,Salma Hayek,/ka1luhF3DrSMEeY4dd5bKZOsPPS.jpg,315162,Kitty Softpaws (voice)
2,3,210172,Harvey Guillén,/yiNBonobPwqMVweB02JWufzp2l9.jpg,315162,Perrito (voice)
3,4,52583,Wagner Moura,/6gcfwvOueJKhDpTP9KLGuWz0Hk4.jpg,315162,Wolf (voice)
4,5,1373737,Florence Pugh,/421cSReX2Fktldac8SyY2k0yLwY.jpg,315162,Goldilocks (voice)


In [11]:
actors_df.to_csv('Data/actors.csv')

# 4. Import DataFrames to Postgres Database
Connect to the local Postgres database and store movies, directors, and actors into the respective tables. Tables within the local Postgres database should already be created before attempting.

In [12]:
# Connect to local database.
protocol = 'postgresql'
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = 5432
database_name = 'project03_group11_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [13]:
# Check for tables.
engine.table_names()

C:\Users\pbhan\AppData\Local\Temp\ipykernel_19292\4178942395.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['movie', 'actor', 'director']

In [14]:
# Store lists of movies, directors, and actors into Postgres as separate tables.
movies_df.to_sql(
    name='movie',
    con=engine,
    if_exists='append',
    index=False
)

858

In [15]:
directors_df.to_sql(
    name='director',
    con=engine,
    if_exists='append',
    index=False
)

855

In [16]:
actors_df.to_sql(
    name='actor',
    con=engine,
    if_exists='append',
    index=False
)

359